In [1]:
from keras.models import Sequential
from keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
import argparse

In [2]:
LEARNING_RATE = 0.001
EPOCHS = 10
BATCH_SIZE = 128

In [3]:
def create_model(width, height, depth, classes):
    model = Sequential()
    inputShape = (height, width, depth)
    
    model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', padding="same", input_shape=inputShape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation("relu"))
    model.add(Dropout(0.5))
    
    model.add(Dense(64))
    model.add(Activation("relu"))
    model.add(Dropout(0.5))
    
    model.add(Dense(classes))
    model.add(Activation("softmax"))

    adam = Adam(lr=LEARNING_RATE)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [4]:
((X_train, y_train), (X_test, y_test)) = mnist.load_data()

# add a channel (i.e., grayscale) dimension to the digits
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))

# scale data to the range of [0, 1]
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

# convert the labels from integers to vectors
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

In [5]:
model = create_model(width=28, height=28, depth=1, classes=10)
model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(X_test, y_test), verbose=1)

Epoch 1/10
469/469 [==============================] - 22s 45ms/step - loss: 0.6850 - accuracy: 0.7749 - val_loss: 0.0952 - val_accuracy: 0.9716
Epoch 2/10
469/469 [==============================] - 20s 44ms/step - loss: 0.2481 - accuracy: 0.9270 - val_loss: 0.0590 - val_accuracy: 0.9819
Epoch 3/10
469/469 [==============================] - 22s 46ms/step - loss: 0.1886 - accuracy: 0.9458 - val_loss: 0.0523 - val_accuracy: 0.9858
Epoch 4/10
469/469 [==============================] - 21s 44ms/step - loss: 0.1548 - accuracy: 0.9550 - val_loss: 0.0427 - val_accuracy: 0.9881
Epoch 5/10
469/469 [==============================] - 20s 43ms/step - loss: 0.1383 - accuracy: 0.9610 - val_loss: 0.0405 - val_accuracy: 0.9885
Epoch 6/10
469/469 [==============================] - 21s 44ms/step - loss: 0.1207 - accuracy: 0.9646 - val_loss: 0.0368 - val_accuracy: 0.9893
Epoch 7/10
469/469 [==============================] - 21s 44ms/step - loss: 0.1098 - accuracy: 0.9681 - val_loss: 0.0345 - val_accuracy:

In [6]:
# evaluate the network
predictions = model.predict(X_test)
print(classification_report(y_test.argmax(axis=1), predictions.argmax(axis=1), target_names=[str(x) for x in lb.classes_]))

313/313 [==============================] - 2s 6ms/step
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       980
           1       0.99      1.00      0.99      1135
           2       0.97      1.00      0.99      1032
           3       1.00      0.98      0.99      1010
           4       0.99      1.00      0.99       982
           5       0.97      0.99      0.98       892
           6       0.99      0.99      0.99       958
           7       0.99      0.98      0.99      1028
           8       0.99      0.99      0.99       974
           9       0.99      0.97      0.98      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000



In [7]:
from tensorflow.keras.models import save_model

save_model(model, 'digit_classifier.h5')